In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Read in the datasets
movies = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python', encoding='utf-8')
reviews = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python', encoding='utf-8')

%matplotlib inline

In [3]:
movies.head()

,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


In [4]:
reviews.head()

,user_id,movie_id,rating,timestamp
0,1,0114508,8,1381006850
1,2,0499549,9,1376753198
2,2,1305591,8,1376742507
3,2,1428538,1,1371307089
4,3,0075314,1,1595468524


#### Explore Data

In [5]:
# Number of Movies
print(f"The number of movies: {movies.shape[0]}")

# Number of Ratings
print(f"The number of ratings: {reviews.shape[0]}")

The number of movies: 37191
The number of ratings: 902957


In [6]:
reviews.isnull().sum()

user_id      0
movie_id     0
rating       0
timestamp    0
dtype: int64

In [7]:
# Number of Different genres
genres = []
for val in movies.genre:
    try:
        genres.extend(val.split('|'))
    except AttributeError:
        pass

genres = set(genres)
print(f"The number of genres is {len(genres)}")

The number of genres is 28


In [8]:
reviews.describe()

,rating
count,902957.000000
mean,7.316013
std,1.852766
min,0.000000
25%,6.000000
50%,8.000000
75%,9.000000
max,10.000000


#### Data Cleaning

In [9]:
# Pull Date if exists
create_date = lambda val: val[-5:-1] if val[-1] == ')' else np.nan

# Apply the Function to Pull the Date
movies['date'] = movies['movie'].apply(create_date)

movies.head()

,movie_id,movie,genre,date
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895
4,0000091,Le manoir du diable (1896),Short|Horror,1896


In [10]:
# Return century of a movie as a dummy column
def add_movie_year(val):
    if val[:2] == yr:
        return 1
    else:
        return 0
    
# Apply Function
for yr in ['18', '19', '20']:
    movies[str(yr)+"00's"] = movies['date'].apply(add_movie_year)

movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0


In [11]:
# Function to Split and Return Values for Columns

def split_genres(val):
    try:
        if val.find(gene) > -1:
            return 1
        else:
            return 0
    except AttributeError:
        return 0
    
# Apply function to Each Genre
for gene in genres:
    movies[gene] = movies['genre'].apply(split_genres)

In [12]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,Horror,News,Western,...,Film-Noir,Talk-Show,Musical,Sci-Fi,Drama,History,Reality-TV,Thriller,Documentary,Adult
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
import datetime

change_timestamp = lambda val: datetime.datetime.fromtimestamp(int(val)).strftime('%Y-%m-%d %H:%M:%S')

reviews['date'] = reviews['timestamp'].apply(change_timestamp)

In [14]:
reviews.head()

,user_id,movie_id,rating,timestamp,date
0,1,0114508,8,1381006850,2013-10-06 02:30:50
1,2,0499549,9,1376753198,2013-08-17 20:56:38
2,2,1305591,8,1376742507,2013-08-17 17:58:27
3,2,1428538,1,1371307089,2013-06-15 20:08:09
4,3,0075314,1,1595468524,2020-07-23 07:12:04


In [15]:
# Now reviews and movies are the final dataframes with the necessary columns
reviews.to_csv('Data/reviews_clean.csv')
movies.to_csv('Data/movies_clean.csv')